# Food ingredient classification model

# Done By: Yousef. A. Alyazouri

# 0.Introduction

### In this project, I have made a ptyorch classification model for predicting up to 51 kind of fruits and vegetables using postman API as an input method

<center><img src="https://domf5oio6qrcr.cloudfront.net/medialibrary/11499/3b360279-8b43-40f3-9b11-604749128187.jpg" alt="Drug Picture" width="1280" height="720"></center><br>

### 🔧 Step 1: Import Required Libraries
We import PyTorch, TorchVision, and image-related libraries. 
`Flask` is only needed if we run this as an API, not in the notebook.

In [ ]:
from flask import Flask, request, jsonify
import torch
from torchvision import models, transforms
from PIL import Image

In [ ]:
app = Flask(__name__)

### 🧠 Step 2: Load the Pretrained Model
We use a ResNet-50 architecture and modify the final layer to classify 51 categories. 
The model is loaded to the appropriate device (CPU or GPU).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 51)  # For 51 fruit/vegetable classes

model.load_state_dict(torch.load("fruits_vegetables_51.pth", map_location=device))
model.to(device)
model.eval()


### 🏷️ Step 3: Class Names
This list maps the model's output index to actual fruit/vegetable names.


In [ ]:
class_names = [
    'Amaranth', 'Apple', 'Banana', 'Beetroot', 'Bell pepper', 'Bitter Gourd',
    'Blueberry', 'Bottle Gourd', 'Broccoli', 'Cabbage', 'Cantaloupe', 'Capsicum',
    'Carrot', 'Cauliflower', 'Chilli pepper', 'Coconut', 'Corn', 'Cucumber',
    'Dragon_fruit', 'Eggplant', 'Fig', 'Garlic', 'Ginger', 'Grapes', 'Jalepeno',
    'Kiwi', 'Lemon', 'Mango', 'Okra', 'Onion', 'Orange', 'Paprika', 'Pear', 'Peas',
    'Pineapple', 'Pomegranate', 'Potato', 'Pumpkin', 'Raddish', 'Raspberry',
    'Ridge Gourd', 'Soy beans', 'Spinach', 'Spiny Gourd', 'Sponge Gourd',
    'Strawberry', 'Sweetcorn', 'Sweetpotato', 'Tomato', 'Turnip', 'Watermelon'
]


### 🖌️ Step 4: Define Image Preprocessing
We resize, convert to tensor, and normalize the image using standard ImageNet values.


In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


### 🖼️ Step 5: Preparing API for uploading on Postman
Load an image in postman using the resulted API


In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    img = Image.open(file.stream)
    img = data_transforms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)
        prediction = class_names[preds[0]]

    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(debug=True)